In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
pd.options.display.max_columns = 35

In [2]:
import pymongo

In [3]:
mc = pymongo.MongoClient()
db = mc['housing-recommender']
training = db['training']

In [4]:
df = pd.DataFrame(list(training.find()))

In [107]:
import os
now = datetime.now()
directory = os.fsencode('/Users/elisereppond/Downloads')

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    month = now.month
    day = now.day
    if month <= 9:
        month = '0{}'.format(month)
    if day <= 9:
        day = '0{}'.format(day)
    if filename.startswith('redfin_{}-{}-{}'.format(now.year,
                                                    month,day)):
        print(filename)
        print(now.month)

FileNotFoundError: [Errno 2] No such file or directory: b'~/Downloads'

In [74]:
from selenium.webdriver import Chrome
for zipcode in df['ZIP'].unique():
    browser = Chrome()
    time = datetime.datetime.now()
    change = datetime.timedelta(seconds=82)
    browser.get('https://www.redfin.com/zipcode/{}'.format(zipcode))
    download_button = browser.find_element_by_css_selector('a#download-and-save')
    download_button.click()
    print(time - change)
#     pd.read_csv('~/Downloads/redfin_{}-{}-{}-{}-{}-{}'.format(time.year, 
#                                                               time.month,
#                                                               time.day,
#                                                               time.hour,
#                                                               time.minute, 
#                                                               time.second))

2018-05-08 14:31:36.519785
2018-05-08 14:31:43.025435


KeyboardInterrupt: 

In [78]:
from datetime import timedelta

In [79]:
time1 = datetime.datetime.time(14,31,37)
time2 = datetime.datetime.time(14,31,43)
time1+change

AttributeError: type object 'datetime.datetime' has no attribute 'datetime'

In [37]:
df_nsea = pd.read_csv('../../data-housing/north-seattle-may-7.csv')

In [6]:
df.head(1)

,ADDRESS,BATHS,BEDS,CITY,DAYS ON MARKET,DESC,HOA/MONTH,LATITUDE,LOCATION,LONGITUDE,LOT SIZE,PRICE,PROPERTY TYPE,SALE TYPE,SQUARE FEET,SQUAREFT,STATE,STATUS,URL,YEAR BUILT,ZIP,_id,ts
0,711 153rd St SW,3.0,4,Lynnwood,3,Gorgeous 4 bed 3 bath Lynnwood home w/ high-en...,0.0,47.859744,Lynnwood,-122.243075,7405.0,750000,Single Family Residential,MLS Listing,3252.0,231.0,WA,Active,http://www.redfin.com/WA/Lynnwood/711-153rd-St...,2016.0,98087,5af0cc9461fbeb7738834c4b,2018-05-07 15:00:40.693


In [37]:
from nltk.tokenize import sent_tokenize

sent_tokens = [sent_tokenize(desc) for desc in df['DESC']]

sent_tokens

[['Gorgeous 4 bed 3 bath Lynnwood home w/ high-end finishes throughout makes this home feel like new.',
  'Kitchen features large island, SS appliances & Quartz counter-tops.',
  'Open & spacious floor plan lets in lots of natural light.',
  'Master bedroom w/ walk-in closet and beautiful 5 piece en suite including beautiful gas fireplace for added comfort & ambiance.',
  'Office/den on main floor.',
  'Fully fenced backyard offers covered patio & garden space.',
  'Close to shopping, parks, transit center & I-5/I-405.'],
 ['Head down the driveway to the privacy of this completely remodeled hidden gem.',
  'Updates include:New roof/gutters, garage door, cabinets, granite counter-top, SS appliances, electric panel, H2O heater, washer/dryer.',
  'All new windows & flooring, new tile in the kitchen & bathroom, and fresh paint both in & out.',
  'Enjoy the multitude of amenities this area has to offer including easy access to the 509,405,I-5,the airport & Light Rail.',
  'Large lot is read

In [39]:
from nltk.tokenize import word_tokenize

tokens = [[word_tokenize(sent) for sent in desc] for desc in sent_tokens]

list(enumerate(tokens))

[(0,
  [['Gorgeous',
    '4',
    'bed',
    '3',
    'bath',
    'Lynnwood',
    'home',
    'w/',
    'high-end',
    'finishes',
    'throughout',
    'makes',
    'this',
    'home',
    'feel',
    'like',
    'new',
    '.'],
   ['Kitchen',
    'features',
    'large',
    'island',
    ',',
    'SS',
    'appliances',
    '&',
    'Quartz',
    'counter-tops',
    '.'],
   ['Open',
    '&',
    'spacious',
    'floor',
    'plan',
    'lets',
    'in',
    'lots',
    'of',
    'natural',
    'light',
    '.'],
   ['Master',
    'bedroom',
    'w/',
    'walk-in',
    'closet',
    'and',
    'beautiful',
    '5',
    'piece',
    'en',
    'suite',
    'including',
    'beautiful',
    'gas',
    'fireplace',
    'for',
    'added',
    'comfort',
    '&',
    'ambiance',
    '.'],
   ['Office/den', 'on', 'main', 'floor', '.'],
   ['Fully',
    'fenced',
    'backyard',
    'offers',
    'covered',
    'patio',
    '&',
    'garden',
    'space',
    '.'],
   ['Close',
    'to'

In [41]:
import string

tokens_lower = [[[thing.lower() for thing in word] for word in sent]
                 for sent in tokens]

In [42]:
from nltk.corpus import stopwords

stopwords_ = set(stopwords.words('english'))

In [46]:
tokens_lower

[[['gorgeous',
   '4',
   'bed',
   '3',
   'bath',
   'lynnwood',
   'home',
   'w/',
   'high-end',
   'finishes',
   'throughout',
   'makes',
   'this',
   'home',
   'feel',
   'like',
   'new',
   '.'],
  ['kitchen',
   'features',
   'large',
   'island',
   ',',
   'ss',
   'appliances',
   '&',
   'quartz',
   'counter-tops',
   '.'],
  ['open',
   '&',
   'spacious',
   'floor',
   'plan',
   'lets',
   'in',
   'lots',
   'of',
   'natural',
   'light',
   '.'],
  ['master',
   'bedroom',
   'w/',
   'walk-in',
   'closet',
   'and',
   'beautiful',
   '5',
   'piece',
   'en',
   'suite',
   'including',
   'beautiful',
   'gas',
   'fireplace',
   'for',
   'added',
   'comfort',
   '&',
   'ambiance',
   '.'],
  ['office/den', 'on', 'main', 'floor', '.'],
  ['fully',
   'fenced',
   'backyard',
   'offers',
   'covered',
   'patio',
   '&',
   'garden',
   'space',
   '.'],
  ['close',
   'to',
   'shopping',
   ',',
   'parks',
   ',',
   'transit',
   'center',
   '&',


In [47]:
punctuation_ = set(string.punctuation)

def filter_tokens(sent):
    return([w for w in sent if not w in stopwords_ and not w in punctuation_])

tokens_filtered = [[filter_tokens(sent) for sent in desc] for desc in tokens_lower]

for sent in tokens_filtered:
    print("--- sentence tokens: {}".format(sent))

--- sentence tokens: [['gorgeous', '4', 'bed', '3', 'bath', 'lynnwood', 'home', 'w/', 'high-end', 'finishes', 'throughout', 'makes', 'home', 'feel', 'like', 'new'], ['kitchen', 'features', 'large', 'island', 'ss', 'appliances', 'quartz', 'counter-tops'], ['open', 'spacious', 'floor', 'plan', 'lets', 'lots', 'natural', 'light'], ['master', 'bedroom', 'w/', 'walk-in', 'closet', 'beautiful', '5', 'piece', 'en', 'suite', 'including', 'beautiful', 'gas', 'fireplace', 'added', 'comfort', 'ambiance'], ['office/den', 'main', 'floor'], ['fully', 'fenced', 'backyard', 'offers', 'covered', 'patio', 'garden', 'space'], ['close', 'shopping', 'parks', 'transit', 'center', 'i-5/i-405']]
--- sentence tokens: [['head', 'driveway', 'privacy', 'completely', 'remodeled', 'hidden', 'gem'], ['updates', 'include', 'new', 'roof/gutters', 'garage', 'door', 'cabinets', 'granite', 'counter-top', 'ss', 'appliances', 'electric', 'panel', 'h2o', 'heater', 'washer/dryer'], ['new', 'windows', 'flooring', 'new', 'tile

In [53]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer

stemmer_porter = PorterStemmer()
tokens_stemporter = [[list(map(stemmer_porter.stem, sent)) for sent in desc] for desc in tokens_filtered]
print("--- sentence tokens (porter): {}".format(tokens_stemporter[0]))

stemmer_snowball = SnowballStemmer('english')
tokens_stemsnowball = [[list(map(stemmer_snowball.stem, sent)) for sent in desc] for desc in tokens_filtered]
print("--- sentence tokens (snowball): {}".format(tokens_stemsnowball[0]))

--- sentence tokens (porter): [['gorgeou', '4', 'bed', '3', 'bath', 'lynnwood', 'home', 'w/', 'high-end', 'finish', 'throughout', 'make', 'home', 'feel', 'like', 'new'], ['kitchen', 'featur', 'larg', 'island', 'ss', 'applianc', 'quartz', 'counter-top'], ['open', 'spaciou', 'floor', 'plan', 'let', 'lot', 'natur', 'light'], ['master', 'bedroom', 'w/', 'walk-in', 'closet', 'beauti', '5', 'piec', 'en', 'suit', 'includ', 'beauti', 'ga', 'fireplac', 'ad', 'comfort', 'ambianc'], ['office/den', 'main', 'floor'], ['fulli', 'fenc', 'backyard', 'offer', 'cover', 'patio', 'garden', 'space'], ['close', 'shop', 'park', 'transit', 'center', 'i-5/i-405']]
--- sentence tokens (snowball): [['gorgeous', '4', 'bed', '3', 'bath', 'lynnwood', 'home', 'w/', 'high-end', 'finish', 'throughout', 'make', 'home', 'feel', 'like', 'new'], ['kitchen', 'featur', 'larg', 'island', 'ss', 'applianc', 'quartz', 'counter-top'], ['open', 'spacious', 'floor', 'plan', 'let', 'lot', 'natur', 'light'], ['master', 'bedroom', 'w

In [77]:
from nltk.util import ngrams

def join_sent_ngrams(input_tokens, n):
    # first add the 1-gram tokens
    ret_list = list(input_tokens)
    
    #then for each n
    for i in range(2,n+1):
        # add each n-grams to the list
        ret_list.extend(['-'.join(tgram) for tgram in ngrams(input_tokens, i)])
    
    return(ret_list)

snowball = SnowballStemmer('english')

In [78]:
sent_tokens = []
word_tokens = []
filtered_tokens = []
for idx, row in df.iterrows():
    sent_tokens.append([sent_tokenize(desc) for desc in row['DESC']])
    word_tokens.append([[word_tokenize(sent.lower()) for sent in desc] for desc in sent_tokens[idx]])
    filtered_tokens.append([filter_tokens(sent) for sent in tokens_lower[idx]])
snowball_lst = [[list(map(snowball.stem, sent)) for sent in desc] for desc in filtered_tokens]

In [79]:
snowball_lst

[[['gorgeous',
   '4',
   'bed',
   '3',
   'bath',
   'lynnwood',
   'home',
   'w/',
   'high-end',
   'finish',
   'throughout',
   'make',
   'home',
   'feel',
   'like',
   'new'],
  ['kitchen',
   'featur',
   'larg',
   'island',
   'ss',
   'applianc',
   'quartz',
   'counter-top'],
  ['open', 'spacious', 'floor', 'plan', 'let', 'lot', 'natur', 'light'],
  ['master',
   'bedroom',
   'w/',
   'walk-in',
   'closet',
   'beauti',
   '5',
   'piec',
   'en',
   'suit',
   'includ',
   'beauti',
   'gas',
   'fireplac',
   'ad',
   'comfort',
   'ambianc'],
  ['office/den', 'main', 'floor'],
  ['fulli', 'fenc', 'backyard', 'offer', 'cover', 'patio', 'garden', 'space'],
  ['close', 'shop', 'park', 'transit', 'center', 'i-5/i-405']],
 [['head', 'driveway', 'privaci', 'complet', 'remodel', 'hidden', 'gem'],
  ['updat',
   'includ',
   'new',
   'roof/gutt',
   'garag',
   'door',
   'cabinet',
   'granit',
   'counter-top',
   'ss',
   'applianc',
   'electr',
   'panel',
   'h2o',

In [80]:
from sklearn import preprocessing

In [81]:
le = preprocessing.LabelEncoder()
le.fit(df['DESC'])

LabelEncoder()

In [122]:
le.transform(df['DESC'])

array([128, 141, 126, 216, 236, 324, 122, 300,  80, 149, 274, 222, 317,
        20, 147,  94, 215, 327, 104, 110, 337, 230, 315, 282, 208,  83,
       325, 131,  31, 198, 127, 278, 235, 174, 146, 186, 234, 244, 192,
       255,  34, 283,  65,  25, 204, 303,  95,  75, 332,  55, 172, 241,
       226, 163, 195, 152,  64, 165, 129, 161, 323, 159, 177, 297,  35,
       153, 148, 344,  77, 108,   9, 185, 275, 171, 314, 307, 302,  85,
       133, 134,   1,   5,  30, 258, 145, 181, 285,  37, 123, 257, 245,
        53,  84,  91, 310, 248, 132,   2, 160, 326, 289, 316, 272, 253,
        54, 183, 292, 176, 229, 180, 121, 247, 202,  63, 162, 223, 239,
       166,  69, 348, 336, 220,  82, 224, 103, 264, 193, 330,  78, 311,
       277,  41, 189, 347, 250, 158, 304,  32, 219,  92, 313, 164, 105,
        19, 299,  12, 308,  33, 339, 259,  47,  62,  23, 335, 251, 184,
        28,  15, 112, 207, 218, 342, 196, 279, 312,  18, 309,  58, 209,
       206, 139,  49, 144, 137,  40,  36,  11, 262,  38, 228, 21

In [85]:
corpus = [row['DESC'] for idx, row in df.iterrows()]

In [121]:
from sklearn.feature_extraction.text import CountVectorizer

tf = CountVectorizer()

document_tf_matrix = tf.fit_transform(corpus).todense()

print(sorted(tf.vocabulary_))
print(document_tf_matrix)

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
document_tfidf_matrix = tfidf.fit_transform(corpus)
print(sorted(tfidf.vocabulary_))
print(document_tfidf_matrix.todense())

document_tfidf_matrix.todense().shape

df['TFIDF'] = pd.Series(np.array(document_tfidf_matrix.todense()).reshape(-1))

df['TFIDF'].value_counts()

In [142]:
y = le.transform(df['DESC'])
# y = df['category'].values
X = df[['DESC']]

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

# data_train, data_test = df[train_index], df[test_index]
tfidf = TfidfVectorizer()
tfidf.fit(X_train['DESC'].values)

X_train_matrix = tfidf.transform(X_train['DESC'].values)

X_test_matrix = tfidf.transform(X_test['DESC'].values)

X_train_matrix.shape

X_train_matrix.todense()

X_test_matrix.shape

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

X_train_matrix.todense().shape

mnb = MultinomialNB(alpha=1)
mnb.fit(X_train_matrix.todense(), y_train)

MultinomialNB(alpha=1, class_prior=None, fit_prior=True)

In [148]:
mnb.predict(X_test_matrix.todense())

array([266, 330, 341, 312, 216, 331,   2, 115, 296, 266, 153,  39, 309,
       219, 230,  15,  39, 151,  39, 146, 153, 283, 248, 337,  38, 325,
       183, 141, 284,  64, 105, 205, 235,  37,  14, 275,  59, 120,  67,
       299, 342, 134, 230, 216, 104, 325, 258, 303, 335, 216,  33, 283,
        50, 207, 153,  25, 347, 151, 284, 331,  69, 149,  19, 216, 104,
       153, 150,  67, 255, 123, 335, 105,  11, 115, 279,  69, 195, 207,
       268, 317, 105, 274, 115,  37, 149,  39, 150, 185])

In [147]:
mnb.classes_

array([  0,   1,   2,   3,   4,   5,   6,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  24,  25,  26,  27,  28,
        29,  30,  31,  33,  37,  38,  39,  40,  41,  44,  45,  48,  50,
        51,  52,  56,  57,  58,  59,  61,  63,  64,  65,  66,  67,  68,
        69,  71,  72,  73,  74,  76,  77,  79,  81,  84,  85,  87,  88,
        90,  91,  92,  93,  94,  95,  96,  97,  99, 100, 101, 102, 104,
       105, 106, 107, 108, 110, 111, 112, 114, 115, 116, 117, 118, 119,
       120, 121, 123, 125, 126, 127, 128, 130, 132, 134, 136, 137, 139,
       140, 141, 142, 143, 144, 146, 147, 149, 150, 151, 152, 153, 154,
       155, 156, 157, 160, 161, 162, 163, 164, 165, 167, 169, 170, 171,
       172, 173, 174, 175, 176, 177, 178, 179, 182, 183, 184, 185, 187,
       191, 192, 194, 195, 196, 197, 199, 200, 201, 202, 203, 205, 206,
       207, 208, 209, 210, 211, 212, 214, 215, 216, 218, 219, 220, 221,
       222, 223, 225, 227, 230, 231, 232, 233, 235, 236, 237, 23

In [149]:
df['DESC'][115]

'Remodeled mid-century rambler surrounded by nature & in Issaquah School District. Secluded location with Cedar River and territorial views yet convenient to all areas via 405, I5, and 167 as well as “back road” thoroughfares – great for commuters! Floor to ceiling windows bring amazing light. Huge deck overlooking Cedar River for year round entertainment. All new finishes: windows, electrical, hardwoods, quartz countertops and much more. Large attached carport.'

K Means Clustering

In [7]:
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
tfidf = TfidfVectorizer(stop_words='english', max_features=50)
tfidf.fit(df.DESC.values)
desc_tfidf = tfidf.transform(df.DESC.values)

In [22]:
km = KMeans(n_clusters=30)
km.fit(desc_tfidf.todense())

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=30, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [32]:
X_test = ['Take in westerly corridor views of the Puget Sound from your front deck and living areas. Imagine adding a second story to capture bigger views of the Sound and Olympic Mountains. This solid 1940s home sits proudly on a large lot with garden areas, private driveway & 2-car garage. In recent years, new water line run to the street, sewer lined and a trench drain system was installed in the basement. A house loved for 80 years by its original owners. Now ready to be reinvented and loved again.', 
          'Opportunity knocks! 1 bed unit in Duncan Place’s coveted 11 stack. Gorgeous floor to ceiling windows & private deck give the unit a sleek modern feel. Bright & sunny w/ open concept kitchen & center island -perfect for entertaining. Upgrades: new flooring, recessed lighting, designer blinds, & fresh paint. In unit laundry & secure parking. Building amenities: roof deck w/ amazing views, secure access & storage unit. EZ commute by bus or future LT Rail. No rental cap & low dues = great investment.']
tfidf_test = TfidfVectorizer(stop_words='english', max_features=50)
tfidf_test.fit(X_test)
test_desc = tfidf_test.transform(X_test)
test_desc
list(km.predict(test_desc))

[6, 16]

In [26]:
df['labels'] = pd.Series(km.labels_)

In [44]:
df[df['labels'] == 20]['DESC']

17     Welcome home! Amazing craftsmanship, open, lig...
35     Natural light filled home w/ 3 bedrooms & full...
54     One of the largest floorpans in Seola Gardens,...
55     Immaculate move-in ready condo at coveted DayB...
104    Bright & spacious 3 bdrm +bonus rooms w/ custo...
145    A great opportunity to live in Wisteria Statio...
159    Presenting Staunton Cove, a quiet community of...
191    Welcome to highly sought after Point Edwards. ...
232    Opportunity knocks!Enjoy a prime location near...
309    Breathtaking Bellevue residence w/ views of La...
329    Cosmetic improvements and updates will make th...
335    Spacious 2002 Craftsman w/ versatile living sp...
343    2 bedroom 1.75 bath condo in desirable Renton ...
Name: DESC, dtype: object

In [45]:
top_features = np.argmax(km.cluster_centers_, axis=1)

In [11]:
top_features

array([44, 38, 31, 33, 38, 35, 44, 19, 38, 25, 37, 19, 39, 48, 26, 38, 24,
       30,  5, 46, 12, 44,  8, 45, 12, 38, 45,  2, 21, 17])

In [156]:
desc_tfidf[:, top_features].todense()

matrix([[ 0.14887418,  0.14887418,  0.        , ...,  0.23534754,
          0.        ,  0.        ],
        [ 0.58577568,  0.19525856,  0.        , ...,  0.        ,
          0.        ,  0.26320303],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.2775148 ,  0.24483353],
        ..., 
        [ 0.        ,  0.        ,  0.18742187, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.1850518 ,  0.        ,  0.15254881, ...,  0.29253888,
          0.        ,  0.        ],
        [ 0.6276003 ,  0.        ,  0.17245564, ...,  0.16535684,
          0.        ,  0.        ]])

In [157]:
for centroid in km.cluster_centers_:
    dense_art = desc_tfidf.todense()
    indices = np.argsort(centroid)[:10]
    #dense_art[:, indices]
    reversed_vocab = {index: word for word, index in tfidf.vocabulary_.items()}
    print([reversed_vocab[index] for index in indices])

['beautiful', 'level', 'car', 'lot', 'light', 'fenced', 'yard', 'living', 'bedroom', 'windows']
['enjoy', 'views', 'unit', 'storage', 'parking', 'lake', 'location', 'bedrooms', 'level', 'access']
['gas', 'access', 'walk', 'views', 'updated', 'unit', 'storage', 'ss', 'spacious', 'parking']
['views', 'unit', 'parking', 'features', 'enjoy', 'closet', 'access', 'deck', 'yard', 'storage']
['lake', 'windows', 'floors', 'unit', 'features', 'storage', 'location', 'granite', 'level', 'light']
['yard', 'master', 'bedrooms', 'suite', 'home', 'deck', 'windows', 'updated', 'bedroom', 'walk']
['unit', 'lake', 'suite', 'area', 'dining', 'windows', 'updated', 'master', 'open', 'granite']
['bedroom', 'windows', 'location', 'fenced', 'bedrooms', 'room', 'space', 'fireplace', 'patio', 'gas']
['unit', 'car', 'access', 'garage', 'open', 'ss', 'parking', 'location', 'closet', 'close']
['beautiful', 'updated', 'unit', 'parking', 'windows', 'enjoy', 'great', 'fenced', 'fireplace', 'access']
['updated', 'area'

In [158]:
reversed_vocab = {index: word for word, index in tfidf.vocabulary_.items()}
[reversed_vocab[index] for index in top_features]

['new',
 'large',
 'room',
 'floor',
 'bath',
 'unit',
 'yard',
 'lake',
 'room',
 'home',
 'beautiful',
 'bath',
 'views',
 'views',
 'parking',
 'lake',
 'walk',
 'windows',
 'great',
 'views',
 'access',
 'updated',
 'space',
 'lot',
 'level',
 'close',
 'area',
 'home',
 'unit',
 'enjoy']

In [160]:
import random

In [161]:
random_samples = np.array(random.sample(list(np.ndenumerate(km.labels_)), 100))

In [163]:
indeces = [item[0][0] for item in random_samples]

centroids = [item[1] for item in random_samples]
stuff = df.iloc[indeces][['PROPERTY TYPE', 'DESC']]

In [165]:
stuff['centroid'] = centroids